<a href="https://colab.research.google.com/github/ChrisBagdon/Citation_Classification/blob/main/tools.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from string import punctuation
from math import log

In [1]:
text = "By clustering with lowly aggressive close kin (King 1989a,b; Viblanc et al. 2010; Arnaud, Dobson & Murie 2012), breeding females may decrease the time/energy cost of maintaining territorial boundaries (Festa-Bianchet & Boag 1982; Murie & Harris 1988), which could ultimately lead to increases in net energy income (TA) or higher allocations in somatic or reproductive functions."

In [27]:
tokens = {}

In [26]:
def tokenize(text, tokens):
  cur_token =""
  for char in text:
    # Check if is apart of token
    if char.isalnum() or char == "'":
      cur_token += char
      continue
    # Check for space
    elif char == " ":
      if len(cur_token) > 0:
        if cur_token in tokens:
          tokens[cur_token] += 1
          cur_token = ""
          continue
        else:
          tokens[cur_token] = 1
          cur_token = ""
          continue
    # Check if punctuation
    else:
      if len(cur_token) > 0:
        if cur_token in tokens:
          tokens[cur_token] += 1
          cur_token = ""
        else:
          tokens[cur_token] = 1
          cur_token = ""
      if char in tokens:
        tokens[char] += 1
        continue
      else:
        tokens[char] = 1
        continue



In [28]:
tokenize(text, tokens)

In [29]:
text_2 = "Ophthalmic symptoms are rare manifestations of the intracranial arachnoid cyst, and include unilateral exophthalmos, visual field abnormality, decreased visual acuity and isolated palsies of the third, fourth and sixth cranial nerves [1–5]."

In [30]:
tokenize(text_2, tokens)

In [31]:
print(tokens.items())

dict_items([('By', 1), ('clustering', 1), ('with', 1), ('lowly', 1), ('aggressive', 1), ('close', 1), ('kin', 1), ('(', 3), ('King', 1), ('1989a', 1), (',', 8), ('b', 1), (';', 3), ('Viblanc', 1), ('et', 1), ('al', 1), ('.', 3), ('2010', 1), ('Arnaud', 1), ('Dobson', 1), ('&', 3), ('Murie', 2), ('2012', 1), (')', 3), ('breeding', 1), ('females', 1), ('may', 1), ('decrease', 1), ('the', 3), ('time', 1), ('/', 1), ('energy', 2), ('cost', 1), ('of', 3), ('maintaining', 1), ('territorial', 1), ('boundaries', 1), ('Festa', 1), ('-', 1), ('Bianchet', 1), ('Boag', 1), ('1982', 1), ('Harris', 1), ('1988', 1), ('which', 1), ('could', 1), ('ultimately', 1), ('lead', 1), ('to', 1), ('increases', 1), ('in', 2), ('net', 1), ('income', 1), ('TA', 1), ('or', 2), ('higher', 1), ('allocations', 1), ('somatic', 1), ('reproductive', 1), ('functions', 1), ('Ophthalmic', 1), ('symptoms', 1), ('are', 1), ('rare', 1), ('manifestations', 1), ('intracranial', 1), ('arachnoid', 1), ('cyst', 1), ('and', 3), ('in

In [ ]:
class naive_bayes:
  def __init__(self):
      self.labels = {}
      self.doc_count = 0
      self.bin_size = 0
  
  def train(self, X, Y):
    for string, label in zip(X, Y):
      # Count instances of labels
      if label not in self.labels:
        self.labels[label] = {'count':1, 'terms':{}}
      else:
        self.labels[label]['count'] += 1
      # Count tokens from document
      tokenize(string, self.labels[label]['terms'])
      # Increase total document count
      self.doc_count += 1
    
    # Tally bin_size for smoothing
    for label, labels_dic in self.labels.items():
      self.bin_size += len(labels_dic['terms'].keys())
    # Calculate class statistics
    for label, labels_dic in self.labels.items():
      # Calculate label prior probability
      self.labels[label]["prior"] = labels_dic['count'] / self.doc_count
      # Save total number of tokens in label + smoothing
      self.labels[label]["term_count"] = sum(labels_dic['terms'].values()+self.bin_size)
      # Calculate probability of each token in label + smoothing
      terms = labels_dic['terms'].keys()
      self.labels[label]["term_probs"] = {term:labels_dic['terms'][term]+1 /
                                          labels_dic["term_count"]
                                           for term in terms}

  def predict(self, X):
    predictions = []

    for string in X:
      tokens = {}
      tokenize(string, tokens)
      probabilities = []
      for label, label_dic in self.labels.items():
        prob = sum(log(label_dic["term_probs"][token])*count if token in label_dic
                       else log(1/label_dic['term_count'])*count 
                       for token, count in tokens.items()) \
               + log(label_dic['prior'])
        probabilities.append((label, prob))
      predictions.append(max(probabilities, key=lambda item:item[1])[0])

    return predictions

      

